In [1]:
from realtime_pollen_calibration import utils
from realtime_pollen_calibration import update_strength_realtime
from realtime_pollen_calibration import update_phenology_realtime
import cfgrib
import scipy.interpolate as interp
import numpy as np
import matplotlib.pyplot as plt
import iconarray
import psyplot.project as psy

In [2]:
def interpolate2(change_tune, ds, coord_stns, epsilon = 100):
    nstns = len(lat_stns)
    stns_points = np.array([[coord_stns[0][i], coord_stns[0][i]] for i in range(nstns)])*np.pi/180
    grid_points = np.array(
        [ds.latitude.values.flatten(), ds.longitude.values.flatten()]
    ).T*np.pi/180
    change_tune_vec = interp.RBFInterpolator(
        stns_points, change_tune-1, kernel="gaussian", epsilon=epsilon, degree = -1, smoothing = 0,
    )(grid_points).reshape(ds.latitude.values.shape) + 1
    return change_tune_vec

def test_change_tune(pollen, file_data, file_data_mod, file_grib, file_grib_tp1):
    data, _, coord_stns, _, _ = utils.read_atab(pollen,
        file_data, file_data_mod
    )
    dst =  cfgrib.open_dataset(
           file_grib,
            encode_cf=("time", "geography", "vertical")
        )
    dstp1 =  cfgrib.open_dataset(
            file_grib_tp1,
            encode_cf=("time", "geography", "vertical")
        )
    for ist in range(data.shape[1]):
        old_tune = utils.get_field_at(dst, pollen + 'tune', coord_stns[ist]).values[0][0]
        new_tune = utils.get_field_at(dstp1, pollen +'tune', coord_stns[ist]).values[0][0]
        print(f'Station {ist} should have change_tune of {new_tune/old_tune}')
        print(f'old value: {old_tune}, new value: {new_tune}')
        print('--------------------------------------------')
        
def test_change_phenol(pollen, file_data, file_grib, file_grib_tp1):
    data, _, coord_stns, _, _ = utils.read_atab(pollen,
        file_data
        )

    dst =  cfgrib.open_dataset(
           file_grib,
            encode_cf=("time", "geography", "vertical")
        )
    dstp1 =  cfgrib.open_dataset(
            file_grib_tp1,
            encode_cf=("time", "geography", "vertical")
        )

    for ist in range(data.shape[1]):
        old_tthre = utils.get_field_at(dst, pollen + 'tthre', coord_stns[ist]).values[0][0]
        new_tthre = utils.get_field_at(dstp1, pollen + 'tthre', coord_stns[ist]).values[0][0]
        print(f'Station {ist} should have change_tthre of {new_tthre - old_tthre}')
        print(f'old value: {old_tthre}, new value: {new_tthre}')
        print('-------------------------------------')
        old_tthrs = utils.get_field_at(dst, pollen + 'tthrs', coord_stns[ist]).values[0][0]
        new_tthrs = utils.get_field_at(dstp1, pollen + 'tthrs', coord_stns[ist]).values[0][0]
        print(f'Station {ist} should have change_tthrs of {new_tthrs - old_tthrs}')
        print(f'old value: {old_tthrs}, new value: {new_tthrs}')
        print('--------------------------------------------')

def get_dif_phenol(pollen, file_data, file_grib, file_grib_tp1, file_out):
    data, _, coord_stns, _, _ = utils.read_atab(pollen,
        file_data
        )

    dst =  cfgrib.open_dataset(
           file_grib,
            encode_cf=("time", "geography", "vertical")
        )
    dstp1 =  cfgrib.open_dataset(
            file_grib_tp1,
            encode_cf=("time", "geography", "vertical")
        )
    dsout =  cfgrib.open_dataset(
            file_out,
            encode_cf=("time", "geography", "vertical")
        )
    nstns = data.shape[1]
    dif_tthre = np.zeros(nstns)
    dif_tthrs = np.zeros(nstns)
    for ist in range(nstns):
        old_tthre = utils.get_field_at(dst, pollen + 'tthre', coord_stns[ist]).values[0][0]
        new_tthre = utils.get_field_at(dstp1, pollen + 'tthre', coord_stns[ist]).values[0][0]
        tthre_fortran = new_tthre - old_tthre
        old_tthre = utils.get_field_at(dst, pollen + 'tthre', coord_stns[ist]).values[0][0]
        new_tthre = utils.get_field_at(dsout, pollen + 'tthre', coord_stns[ist]).values[0][0]
        tthre_py = new_tthre - old_tthre
        dif_tthre[ist] = tthre_fortran - tthre_py
        
        old_tthrs = utils.get_field_at(dst, pollen + 'tthrs', coord_stns[ist]).values[0][0]
        new_tthrs = utils.get_field_at(dstp1, pollen + 'tthrs', coord_stns[ist]).values[0][0]
        tthrs_fortran = new_tthrs - old_tthrs
        old_tthrs = utils.get_field_at(dst, pollen + 'tthrs', coord_stns[ist]).values[0][0]
        new_tthrs = utils.get_field_at(dsout, pollen + 'tthrs', coord_stns[ist]).values[0][0]
        tthrs_py = new_tthrs - old_tthrs
        dif_tthrs[ist] = tthrs_fortran - tthrs_py
    return dif_tthre, dif_tthrs

def get_dif_strength(pollen, file_data, file_grib, file_grib_tp1, file_out):
    data, _, coord_stns, _, _ = utils.read_atab(pollen,
        file_data
        )

    dst =  cfgrib.open_dataset(
           file_grib,
            encode_cf=("time", "geography", "vertical")
        )
    dstp1 =  cfgrib.open_dataset(
            file_grib_tp1,
            encode_cf=("time", "geography", "vertical")
        )
    dsout =  cfgrib.open_dataset(
            file_out,
            encode_cf=("time", "geography", "vertical")
        )
    nstns = data.shape[1]
    dif_tune = np.zeros(nstns)
    for ist in range(nstns):
        old_tune = utils.get_field_at(dst, pollen + 'tune', coord_stns[ist]).values[0][0]
        new_tune = utils.get_field_at(dstp1, pollen + 'tune', coord_stns[ist]).values[0][0]
        tune_fortran = new_tune / old_tune
        old_tune = utils.get_field_at(dst, pollen + 'tune', coord_stns[ist]).values[0][0]
        new_tune = utils.get_field_at(dsout, pollen + 'tune', coord_stns[ist]).values[0][0]
        tune_py = new_tune / old_tune
        dif_tune[ist] = tune_fortran - tune_py
    return dif_tune

In [ ]:
#### ALNU TEST CASE (COSMO KENDA RUN)

file_data = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_measured_values_2022020805.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_modelled_values_2022022207.atab'
file_grib = '/scratch/gvanpary/wd/22022206_325/lm_fine_emermet/lfsf00000000'
file_out = '/scratch/gvanpary/pollen/data/output/COSMO_2022022208_ALNUtune'
update_strength_realtime.update_strength_realtime(file_data, file_data_mod, file_grib, file_out, True)



In [ ]:
#### ALNU TEST CASE

file_data = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_measured_values_2022020805.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_modelled_values_2022022207.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022022207_ALNUtune'
file_out = '/scratch/gvanpary/pollen/data/output/2022022208_ALNUtune_2'
update_strength_realtime.update_strength_realtime(file_data, file_data_mod, file_grib, file_out, False)


'''# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022022208_ALNUtune'
#test_change_tune('ALNU', file_data, file_data_mod, file_grib, file_grib_tp1)
test_change_tune('ALNU', file_data, file_data_mod, file_grib, file_out)
dif_tune = get_dif_strength('ALNU', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tune)'''

In [ ]:
# Get Distance Matrix between each station (currently 9x9 symmetric matrix)
pollen = 'ALNU'
file_data =  '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_measured_values_2022020805.atab'
data, _, coord_stns, _, _ = utils.read_atab(pollen,
        file_data
        )
nstns = data.shape[1]
dist = np.zeros((nstns, nstns))
eps = 1e-10
for ist1 in range(nstns):
    for ist2 in range(nstns):
            diff_lon = (
                (coord_stns[ist1][1] - coord_stns[ist2][1] + eps)
                * np.pi
                / 180
                * np.cos(coord_stns[ist1][0] * np.pi / 180)
            )
            diff_lat = (coord_stns[ist1][0] - coord_stns[ist2][0]) * np.pi / 180
            dist[ist1, ist2] = np.sqrt(
                diff_lon ** 2 + diff_lat ** 2
            )
            
print(dist)
change_test = [2,1,1,1,1,5,1,1,1]
effective_change = np.matmul(1/dist, change_test)/np.matmul(1/dist, np.ones(9))
print(effective_change)

In [ ]:
lat_stns2 = np.zeros(nstns)
lon_stns2 = np.zeros(nstns)
for ist in range(nstns):
    lat_stns2[ist] = utils.get_field_at(dst, 'ALNUtune',coord_stns[ist]).latitude.values[0][0]
    lon_stns2[ist] = utils.get_field_at(dst, 'ALNUtune',coord_stns[ist]).longitude.values[0][0]
coord_stns2 = list(zip(lat_stns2, lon_stns2))
print(coord_stns2)
print(coord_stns)

In [ ]:
dstp1 =  cfgrib.open_dataset(
            file_grib_tp1,
            encode_cf=("time", "geography", "vertical")
        )
# auxiliary plotting variables
lonmin = np.amin(dstp1.longitude)
lonmax = np.amax(dstp1.longitude)
latmin = np.amin(dstp1.latitude)
latmax = np.amax(dstp1.latitude)
plot1 = dstp1.psy.plot.mapplot(
    name="CORYtune",
    title="Alder Tuning Factor ",
    titlesize=15,
    map_extent = [lonmin, lonmax, latmin, latmax],
    bounds = {'method': 'minmax', 'N':100},
    lakes=True,
    borders=True,
    rivers=True,
    cticksize=8,
    clabel="Tuning Factor []",
    grid_labelsize=8,
    projection='robin',
    cmap='RdBu_r', xgrid = False, ygrid = False)
plot1.show()

In [ ]:
dstp1 =  cfgrib.open_dataset(
            file_out,
            encode_cf=("time", "geography", "vertical")
        )
# auxiliary plotting variables
lonmin = np.amin(dstp1.longitude)
lonmax = np.amax(dstp1.longitude)
latmin = np.amin(dstp1.latitude)
latmax = np.amax(dstp1.latitude)
plot1 = dstp1.psy.plot.mapplot(
    name="CORYtune",
    title="Alder Tuning Factor ",
    titlesize=15,
    map_extent = [lonmin, lonmax, latmin, latmax],
    bounds = {'method': 'minmax', 'N':100},
    lakes=True,
    borders=True,
    rivers=True,
    cticksize=8,
    clabel="Tuning Factor []",
    grid_labelsize=8,
    projection='robin',
    cmap='RdBu_r', xgrid = False, ygrid = False)
plot1.show()

In [ ]:
#### BETU TEST CASE

file_data = '/scratch/gvanpary/pollen/data/atabs/betu_pollen_measured_values_2022032309.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/betu_pollen_modelled_values_2022032309.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022032309_BETUtune'
file_out = '/scratch/gvanpary/pollen/data/output/2022032310_BETUtune'
update_strength_realtime.update_strength_realtime(file_data, file_data_mod, file_grib, file_out, True)

# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022032310_BETUtune'
#test_change_tune('BETU', file_data, file_data_mod, file_grib, file_grib_tp1)
test_change_tune('BETU', file_data, file_data_mod, file_grib, file_out)
dif_tune = get_dif_strength('BETU', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tune)

In [ ]:
#### CORY TEST CASE

file_data = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_measured_values_2022021110.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_modelled_values_2022021110.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021110_CORYtune'
file_out = '/scratch/gvanpary/pollen/data/output/2022021110_CORYtune'
update_strength_realtime.update_strength_realtime(file_data, file_data_mod, file_grib, file_out, True)

# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021111_CORYtune'
#test_change_tune('CORY', file_data, file_data_mod, file_grib, file_grib_tp1)
test_change_tune('CORY', file_data, file_data_mod, file_grib, file_out)
dif_tune = get_dif_strength('CORY', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tune)

In [ ]:
#### POAC TEST CASE

file_data = '/scratch/gvanpary/pollen/data/atabs/poac_pollen_measured_values_2022051308.atab'
file_data_mod = '/scratch/gvanpary/pollen/data/atabs/poac_pollen_modelled_values_2022051308.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022051308_POACtune'
file_out = '/scratch/gvanpary/pollen/data/output/2022051309_POACtune'
update_strength_realtime.update_strength_realtime(file_data, file_data_mod, file_grib, file_out, True)

# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022051309_POACtune'
#test_change_tune('POAC', file_data, file_data_mod, file_grib, file_grib_tp1)
test_change_tune('POAC', file_data, file_data_mod, file_grib, file_out)
dif_tune = get_dif_strength('POAC', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tune)

In [ ]:
########## PHENOLOGY PART ################
#### ALNU TEST CASE
file_data = '/scratch/gvanpary/pollen/data/atabs/alnu_pollen_measured_values_2022020805.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022020805_ALNUtthrs_tthre'
file_out = '/scratch/gvanpary/pollen/data/output/2022020805_ALNUtthrs_tthre'
update_phenology_realtime.update_phenology_realtime(file_data, file_grib, file_out, False)

'''# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022020806_ALNUtthrs_tthre'
#test_change_phenol('ALNU', file_data, file_grib, file_grib_tp1)
#test_change_phenol('ALNU', file_data, file_grib, file_out)

dif_tthre, dif_tthrs = get_dif_phenol('ALNU', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tthre)
print(dif_tthrs)'''

In [ ]:
#### BETU TEST CASE
file_data = '/scratch/gvanpary/pollen/data/atabs/betu_pollen_measured_values_2022031814.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022031814_BETUtthrs_tthre'
file_out = '/scratch/gvanpary/pollen/data/output/2022031814_BETUtthrs_tthre'
update_phenology_realtime.update_phenology_realtime(file_data, file_grib, file_out, False)

# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022031815_BETUtthrs_tthre'
#test_change_phenol('BETU', file_data, file_grib, file_grib_tp1)
#test_change_phenol('BETU', file_data, file_grib, file_out)

dif_tthre, dif_tthrs = get_dif_phenol('BETU', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tthre)
print(dif_tthrs)

In [ ]:
#### CORY TEST CASE
file_data = '/scratch/gvanpary/pollen/data/atabs/cory_pollen_measured_values_2022021014.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021014_CORYtthrs_tthre'
file_out = '/scratch/gvanpary/pollen/data/output/2022021014_CORYtthrs_tthre'
update_phenology_realtime.update_phenology_realtime(file_data, file_grib, file_out, True)

# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022021015_CORYtthrs_tthre'
#test_change_phenol('CORY', file_data, file_grib, file_grib_tp1)
#test_change_phenol('CORY', file_data, file_grib, file_out)

dif_tthre, dif_tthrs = get_dif_phenol('CORY', file_data, file_grib, file_grib_tp1, file_out)
print(dif_tthre)
print(dif_tthrs)

In [4]:
#### POAC TEST CASE
file_data = '/scratch/gvanpary/pollen/data/atabs/poac_pollen_measured_values_2022081014.atab'
file_grib = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022081014_POACsaisl'
file_out = '/scratch/gvanpary/pollen/data/output/2022081014_POACsaisl'
update_phenology_realtime.update_phenology_realtime(file_data, file_grib, file_out, True)

# needed for testing
file_grib_tp1 = '/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022081015_POACsaisl'

Station n°0 has 0.0 missing values
Station n°1 has 0.0 missing values
Station n°2 has 0.0 missing values
Station n°3 has 0.0 missing values
Station n°4 has 0.0 missing values
Station n°5 has 0.0 missing values
Station n°6 has 0.0 missing values
Station n°7 has 0.0 missing values
Station n°8 has 0.0 missing values
Station n°9 has 0.0 missing values
Station n°10 has 0.0 missing values


KeyError: 'POACtthrs'

In [5]:
dst =  cfgrib.open_dataset(
           "/scratch/gvanpary/pollen/data/grib2_files_cosmo1e/laf2022081014_POACsaisl",
            encode_cf=("time", "geography", "vertical")
        )
dst

<xarray.Dataset>
Dimensions:            (y: 786, x: 1170)
Coordinates:
    time               datetime64[ns] ...
    step               timedelta64[ns] ...
    surface            float64 ...
    latitude           (y, x) float64 ...
    longitude          (y, x) float64 ...
    valid_time         datetime64[ns] ...
    heightAboveGround  float64 ...
Dimensions without coordinates: y, x
Data variables:
    POACsaisl          (y, x) float32 ...
    POACctsum          (y, x) float32 ...
    POACsaisn          (y, x) float32 ...
    T_2M               (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lssw
    GRIB_centreDescription:  Zurich
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Zurich
    history:                 2023-01-17T17:21 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
change_tune = utils.get_change_tune(pollen_types[ipollen], array, array_mod, ds, lat_stns, lon_stns, istation_mod)


change_tune_vec = interpolate2(change_tune, ds, lat_stns, lon_stns)
tune_vec_OLD = utils.interpolate(change_tune, ds, 'ALNUtune', lat_stns, lon_stns, method= 'multiply', ipollen = 0)

Nstations = len(lon_stns)
tune_OLD = np.zeros(Nstations)
tune_next = np.zeros(Nstations)
for istation in range(Nstations):
    tune_OLD[istation] = utils.get_field_at(ds, 'ALNUtune', lat_stns[istation], lon_stns[istation])
    tune_next[istation] = utils.get_field_at(ds2,'ALNUtune', lat_stns[istation], lon_stns[istation])
change_tune_2 = tune_next/tune_OLD
change_tune_vec_2 = interpolate2(change_tune_2, ds, lat_stns, lon_stns)
tune_vec_OLD_2 = utils.interpolate(change_tune_2, ds, 'ALNUtune', lat_stns, lon_stns, method= 'multiply', ipollen = 0)
#plot_tunevec(ds, 'ALNUtune', change_tune_vec, lat_stns, lon_stns)